In [ ]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp
from genpen.utils import Paper
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
import vpype
from skimage import io
from pathlib import Path

from sklearn.preprocessing import minmax_scale
from skimage import feature
from skimage import exposure

from skimage import filters
from skimage.color import rgb2gray
from skimage.transform import rescale, resize, downscale_local_mean
from skimage.morphology import disk

In [ ]:
def local_angle(dx, dy):
    """Calculate the angles between horizontal and vertical operators."""
    return np.mod(np.arctan2(dy, dx), np.pi)

In [ ]:
image_path= '/home/naka/art/raster_images/blood_red_sunset.png'
paper_size:str = '14x11 inches'
border:float=25  # mm
image_rescale_factor:float=0.3
smooth_disk_size:int=2
hist_clip_limit=0.1
hist_nbins=22
intensity_min=0.05
intensity_max=1.
hatch_spacing_min=0.3  # mm
hatch_spacing_max=1.2  # mm
pixel_width=1  # mm
pixel_height=1  # mm
angle_jitter='ss.norm(scale=8).rvs'  # degrees
# angle_jitter='0'
pixel_rotation='0'  # degrees
merge_tolerances=[0.1, 0.2, 0.3]  # mm
simplify_tolerances=[0.05, 0.1, ]  # mm
savedir='/home/naka/art/plotter_svgs'

In [ ]:
# make page
paper = Paper(paper_size)
drawbox = paper.get_drawbox(border)

# load
img =  rgb2gray(io.imread(Path(image_path)))
img_rescale = np.clip(rescale(img, image_rescale_factor), 0, 1)

img_renorm = exposure.equalize_adapthist(img_rescale, clip_limit=hist_clip_limit, nbins=hist_nbins)
# img_renorm = img_rescale

In [ ]:
plt.imshow(img_renorm)

In [ ]:
# calc dominant angle
selem = disk(smooth_disk_size)
filt_img = filters.rank.mean(img_renorm, selem)
angle_farid = local_angle(filters.farid_h(filt_img), filters.farid_v(filt_img))

# make pixel polys
prms = []
for y, row in tqdm(enumerate(img_renorm)):
    for x, intensity in enumerate(row):

        p = gp.centered_box(Point(x, y), width=pixel_width, height=pixel_height)
        a = np.degrees(angle_farid[y, x])
        prm = {
            'geometry':p,
            'x':x,
            'y':y,
            'raw_pixel_width':pixel_width,
            'raw_pixel_height':pixel_height,
            'intensity': intensity,
            'angle':a,
            'group': 'raw_hatch_pixel',

        }
        prms.append(prm)
raw_hatch_pixels = geopandas.GeoDataFrame(prms)

In [ ]:
#  rescale polys to fit in drawbox
bbox = box(*raw_hatch_pixels.total_bounds)
_, transform = gp.make_like(bbox, drawbox, return_transform=True)
A = gp.AffineMatrix(**transform)
scaled_hatch_pixels = raw_hatch_pixels.copy()
scaled_hatch_pixels['geometry'] = scaled_hatch_pixels.affine_transform(A.A_flat)

In [ ]:
example_height = gp.get_height(scaled_hatch_pixels.loc[0, 'geometry'])
example_width = gp.get_width(scaled_hatch_pixels.loc[0, 'geometry'])
print(f'pixel size = {example_width:.2}x{example_height:.2}mm')
scaled_hatch_pixels['scaled_pixel_height'] = example_height
scaled_hatch_pixels['scaled_pixel_width'] = example_width

In [ ]:
scaled_hatch_pixels['scaled_pixel_min_dim'] = scaled_hatch_pixels.loc[:, ['scaled_pixel_height', 'scaled_pixel_width']].min(axis=1)

In [ ]:
# distributions etc
angle_jitter_gen = gp.make_callable(eval(angle_jitter))
pixel_rotation_gen = gp.make_callable(eval(pixel_rotation))


scaled_hatch_pixels['angle_jitter'] = angle_jitter_gen(len(scaled_hatch_pixels))
scaled_hatch_pixels['hatch_angle'] = scaled_hatch_pixels['angle'] + scaled_hatch_pixels['angle_jitter']
scaled_hatch_pixels['pixel_rotation'] = pixel_rotation_gen(len(scaled_hatch_pixels))

In [ ]:
intensity_min = 0.
intensity_max = 1

In [ ]:
hatch_spacing_min = 0.2

In [ ]:
hatch_spacing_max = 0.75

In [ ]:
spacing_func = functools.partial(np.interp, xp=[intensity_min, intensity_max], fp=[hatch_spacing_max, hatch_spacing_min, ])
scaled_hatch_pixels['spacing'] = spacing_func(1 - scaled_hatch_pixels['intensity'])

In [ ]:
filt_scaled_hatch_pixels = scaled_hatch_pixels.query('spacing < scaled_pixel_min_dim')

In [ ]:
new_rows = []
for i, row in tqdm(filt_scaled_hatch_pixels.iterrows(), total=len(filt_scaled_hatch_pixels)):
    r = row.copy()
    p = r['geometry']
    if abs(r['pixel_rotation']) > np.finfo(float).eps:
        p = sa.rotate(p, r['pixel_rotation'])
    f = gp.hatchbox(p, spacing=r['spacing'], angle=r['hatch_angle'])
    r['geometry'] = f
    new_rows.append(r)

fills = geopandas.GeoDataFrame(new_rows)
fills = fills[fills.length > 0]
fill_layer = gp.merge_LineStrings(fills.geometry)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.stroke(1)
sk.geometry(fill_layer)
sk.display()

In [ ]:
merge_tolerances=[0.05, 0.1, 0.2, 0.3, 0.4, 0.5,] # mm
# simplify_tolerances=[0.05, 0.1,] # mm

In [ ]:
for tolerance in merge_tolerances:
    sk.vpype(f'linemerge --tolerance {tolerance}mm')

for tolerance in simplify_tolerances:
    sk.vpype(f'linesimplify --tolerance {tolerance}mm')
sk.display()

In [ ]:
sk.vpype('linesort')

In [ ]:
import fn

In [ ]:
plot_id = fn.new_plot_id()

In [ ]:
savedir='/home/naka/art/plotter_svgs'

In [ ]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)

In [ ]:
def vsketch_to_shapely(sketch):
    return [[LineString([Point(pt.real, pt.imag) for pt in lc]) for lc in layer] for layer in sketch.document.layers.values()]

In [ ]:
layer = sk.document.layers[1]

In [ ]:
mls = gp.make_like(MultiLineString([LineString([Point(pt.real, pt.imag) for pt in lc]) for lc in layer]), drawbox)

In [ ]:
ds = [150 - ls.distance(drawbox.centroid) for ls in mls]

In [ ]:
ds = np.array(ds) ** 0.5
ds = ds/ ds.sum()

In [ ]:
frac_keep = 0.85
n_keep = int(frac_keep * len(mls))
pmls = MultiLineString(list(np.random.choice(mls, size=n_keep, replace=False,
#                                              p=ds
                                            )))
rlayers = [pmls]

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.08mm')
for i, layer in enumerate(rlayers):
    sk.stroke(i+1)
    sk.geometry(layer)

for tolerance in [0.05, 0.1, 0.2,  0.3, 0.5, 0.7]:
    sk.vpype(f'linemerge --tolerance {tolerance}mm')
    sk.vpype('linesimplify --tolerance 0.1 linesort')

sk.display(color_mode='layer')

In [ ]:
import fn

In [ ]:
plot_id = fn.new_plot_id()

In [ ]:
savedir='/home/naka/art/plotter_svgs'

In [ ]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)